In [120]:
import pandas as pd
import numpy as np
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 

from textblob import TextBlob
from nltk.corpus import stopwords

warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

data = pd.read_csv('Tweets3.csv')

In [121]:
!pip install textblob
import textblob

In [122]:
from textblob import TextBlob
from nltk.corpus import stopwords



In [123]:

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [124]:
data = pd.read_csv('Tweets3.csv')
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['text'], "@[\w]*")

In [125]:
data['tidy_tweet'] = data['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
#data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [126]:
stop_words2= ['https','covid']

for i in range(len(data['tidy_tweet'])):
    data['tidy_tweet'][i]= data['tidy_tweet'][i].lower()
    query= data['tidy_tweet'][i]
    querywords = query.split()
    #querywords  = [word for word in querywords if word.lower() not in stop_words]
    resultwords  = [word for word in querywords if word.lower() not in stop_words2]
    result = ' '.join(resultwords)
    data['tidy_tweet'][i]= result
    

data.head()

C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Sr_No,date,text,username,tidy_tweet
0,0,2020-06-12 12:06:32+00:00,Government mulling to include sudden loss of t...,TheHansIndiaWeb,government mulling to include sudden loss of t...
1,1,2020-06-12 12:04:49+00:00,I know what is common THESE COUNTRIES TRIED SO...,sudarshansvvs,i know what is common these countries tried so...
2,2,2020-06-12 12:03:30+00:00,"To effectively battle Covid-19, India must do ...",DesiDisaNews,to effectively battle india must do much more ...
3,3,2020-06-12 11:59:53+00:00,You should also seek their expertise in handli...,mizozeitgeist,you should also seek their expertise in handli...
4,4,2020-06-12 11:57:10+00:00,"I had called Air India customer care, but was ...",khanra_samapan,i had called air india customer care but was i...


In [127]:
pol=[]
for i in range(len(data['tidy_tweet'])):
      blob = TextBlob(data['tidy_tweet'][i])
      Sentiment = blob.sentiment    
      polarity = Sentiment.polarity
      subjectivity = Sentiment.subjectivity
      pol.append(polarity)

In [128]:
data['polarity']=pol
data.head()

,Sr_No,date,text,username,tidy_tweet,polarity
0,0,2020-06-12 12:06:32+00:00,Government mulling to include sudden loss of t...,TheHansIndiaWeb,government mulling to include sudden loss of t...,0.000000
1,1,2020-06-12 12:04:49+00:00,I know what is common THESE COUNTRIES TRIED SO...,sudarshansvvs,i know what is common these countries tried so...,-0.195833
2,2,2020-06-12 12:03:30+00:00,"To effectively battle Covid-19, India must do ...",DesiDisaNews,to effectively battle india must do much more ...,0.550000
3,3,2020-06-12 11:59:53+00:00,You should also seek their expertise in handli...,mizozeitgeist,you should also seek their expertise in handli...,0.500000
4,4,2020-06-12 11:57:10+00:00,"I had called Air India customer care, but was ...",khanra_samapan,i had called air india customer care but was i...,0.237500


In [129]:
data['label'] = np.where(data['polarity'] >= 0 , 1, 0)
data.head()

,Sr_No,date,text,username,tidy_tweet,polarity,label
0,0,2020-06-12 12:06:32+00:00,Government mulling to include sudden loss of t...,TheHansIndiaWeb,government mulling to include sudden loss of t...,0.000000,1
1,1,2020-06-12 12:04:49+00:00,I know what is common THESE COUNTRIES TRIED SO...,sudarshansvvs,i know what is common these countries tried so...,-0.195833,0
2,2,2020-06-12 12:03:30+00:00,"To effectively battle Covid-19, India must do ...",DesiDisaNews,to effectively battle india must do much more ...,0.550000,1
3,3,2020-06-12 11:59:53+00:00,You should also seek their expertise in handli...,mizozeitgeist,you should also seek their expertise in handli...,0.500000,1
4,4,2020-06-12 11:57:10+00:00,"I had called Air India customer care, but was ...",khanra_samapan,i had called air india customer care but was i...,0.237500,1


In [130]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], 
                                                    data['label'], 
                                                    random_state=0)

In [131]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import roc_auc_score

def test1():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
  
    Nb= BernoulliNB(alpha=0.13) 
    a= Nb.fit(X_train_vectorized,y_train)
    b=a.predict(vect.transform(X_test))
    c= roc_auc_score(y_test,b)
    
    return c

In [132]:
test1()

0.7233186454001944

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
def test2():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    
    Nb= MultinomialNB(alpha=0.1) 
    a= Nb.fit(X_train_vectorized,y_train)
    b=a.predict(vect.transform(X_test))
    c= roc_auc_score(y_test,b)
    

    
    return c

In [111]:
test2()

0.6519270154533606

In [112]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [113]:
from sklearn.svm import SVC

def test3():
    vectorizer = TfidfVectorizer(min_df=3)
    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_with_length = add_feature(X_train_transformed, X_train.str.len())

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_with_length = add_feature(X_test_transformed, X_test.str.len())

    clf = SVC(C=10000)

    clf.fit(X_train_transformed_with_length, y_train)

    y_predicted = clf.predict(X_test_transformed_with_length)
    
    return roc_auc_score(y_test, y_predicted)

In [114]:
test3()

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.6681664908252136

In [115]:
from sklearn.linear_model import LogisticRegression

def test4():
    import re

    vect = TfidfVectorizer(min_df=3).fit(X_train)

    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized= vect.transform(X_test)
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.len())
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda row: len(re.findall(r'(\d)', row))))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.len())
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda row: len(re.findall(r'(\d)', row))))
    model = LogisticRegression(C=10000)
    fit=model.fit(X_train_vectorized, y_train)

    predictions = model.predict(X_test_vectorized)
    
    
    return roc_auc_score(y_test,predictions)

In [116]:
test4()

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7379755476944081

In [117]:
def test5():
    from sklearn.linear_model import LogisticRegression
    
    import re

    vect = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb').fit(X_train)

    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized= vect.transform(X_test)
    X_train_vectorized = add_feature(X_train_vectorized, X_train.str.len())
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda row: len(re.findall(r'(\d)', row))))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.str.len())
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda row: len(re.findall(r'(\d)', row))))
    X_train_vectorized = add_feature(X_train_vectorized, X_train.apply(lambda row: len(re.findall(r'(\W)', row))))
    X_test_vectorized = add_feature(X_test_vectorized, X_test.apply(lambda row: len(re.findall(r'(\W)', row))))
    
    
    model=LogisticRegression(C=100)
    
    fit=model.fit(X_train_vectorized, y_train)
    

    predictions = model.predict(X_test_vectorized)
    
    feature_names = np.array(vect.get_feature_names()+['length_of_doc', 'digit_count', 'non_word_char_count'])

    
    sorted_coef_index = model.coef_[0].argsort()
    a=feature_names[sorted_coef_index[:10]]
    b=feature_names[sorted_coef_index[:-11:-1]]
    c=list(a)
    d=list(b)
    
    f=roc_auc_score(y_test,predictions)
    return f



In [118]:
test5()

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.6906845404331906,
 ['ad ', ' fai', ' wors', 'wors', ' fail', 'ng', ' d', 'ery', 'ta ', 'du'],
 ['ne ', 'lle', ' fi', 'id ', 'll', 'sit', 'na', 'tte', 'u ', 'fi'])